<a href="https://colab.research.google.com/github/fleckc/visSP23/blob/main/ps2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import urllib
import os
from google.colab import files
import time, os, sys, re
import pandas_datareader as pdr
from pandas_datareader import wb

Looking at NJ population, voting habits, and migration. This uses election turnout results from 2000, the 2000 census population counts, and the 2000 census migration patterns in order to retain year similarity. Here are the data sources:

https://www.state.nj.us/state/elections/election-information-ballots-cast.shtml 

(scroll down to 2000 to download the pdf, which I converted to xlsx/csv; it is in a set of cells so it easily converted)

https://www.icpsr.umich.edu/web/ICPSR/studies/13569/

https://www.icpsr.umich.edu/web/ICPSR/studies/13885/


In [4]:
njvote=pd.read_csv('https://github.com/fleckc/visSP23/raw/main/NJ2000ballots.csv')

In [5]:
njpop=pd.read_csv('https://github.com/fleckc/visSP23/raw/main/njpop2000.csv')

In [6]:
njmigrate=pd.read_fwf('https://github.com/fleckc/visSP23/raw/main/countymigrationinflow.txt')

In [ ]:
njvote.head()

In [ ]:
njvote=njvote.rename(columns={'COUNTY': 'County'})
njvote #renaming for easier merge later

In [ ]:
njpop.head()

In [ ]:
countiesd = {'County': ['Atlantic', 'Bergen', 'Burlington', 'Camden', 'Cape May', 'Cumberland', 'Essex', 'Gloucester', 'Hudson', 'Hunterdon', 'Mercer', 'Middlesex', 'Monmouth', 'Morris', 'Ocean', 'Passaic', 'Salem', 'Somerset', 'Sussex', 'Union', 'Warren']}
counties = pd.DataFrame(countiesd)
counties #creating a dataframe (from a dict) for the list of counties to "fix" unmatched county name formats (Atlantic vs Atlantic County)

In [ ]:
njpop=njpop.drop(njpop.index[21:31])
njpop=njpop.drop(['County'], axis=1)
njpop=counties.join(njpop, how="outer")
njpop #dropping extraneous info and renaming counties column for easier merge later

In [ ]:
njmigrate.head()

In [13]:
njmigrate=njmigrate.rename(columns={'01001': 'StateCounty2000',
                 '01003': 'StateCounty1995',
                 '87': 'InflowMigration'})
#renaming columns - note will eliminate some data in this instance but none we will use

In [14]:
njmigratenjonly=njmigrate.query('StateCounty2000 > 34000 & StateCounty2000 < 35000')
#eliminating FIPS codes/counties that are not NJ

In [ ]:
njmigratenjonly.head()

In [16]:
njmigrateonly2=njmigratenjonly.groupby('StateCounty2000')['InflowMigration'].sum()
njmigrateonly2=pd.DataFrame(data=njmigrateonly2)
njmigrateonly2=njmigrateonly2.reset_index(drop="true")
#not really what I wanted but it'll work in the end

In [ ]:
njmigrateonly2.head()

In [18]:
njmerge1 = counties.join(njmigrateonly2, how="outer")
#putting county names to the total migration inflows

In [ ]:
njmerge1.head()

In [20]:
njmerge2=pd.merge(njmerge1, njvote, how="outer" )
#merging migration and turnout

In [ ]:
njmerge2.head()

In [22]:
njmerge3= pd.merge(njpop, njmerge2, how="outer", on='County')
#merging everything together into one dataset with common county naming convention

In [ ]:
njmerge3